# Prediction of restaurant ratings: Main Notebook 

**by Team Tissot: Laux David, Randin Samuel, Solonin Maksim, Vivarié Romin**. 

The business problem we chose to study is the prediction of restaurants ratings. Having notice that reviews being often biased, we wanted to adress a data mining and machine learning problem to analyse the impact of the different variables on restaurant ratings. Our aim is to study the potential correlations of features for a restaurant's rating and the comparison of different technics.

Our first challenge was to select a Data set with enough features and variables to adress the problem. After evaluating various options we chose a Yelp dataset. 
Yelp gives access to their dataset (https://www.yelp.com/dataset/challenge) for the USA and Canada. The website offers ten awards of $5'000 to the best solutions proposed by students for the following challenges:  
- Photo classification  
- Natural language processing  
- Sentiment Analysis and graph mining  

This set includes information about local businesses in 10 metropolitan areas across 2 countries. Round 13 of the challenge was launched on January 15, 2019 and will run through December 31, 2019. 

After downloading the data, we had to convert it to a csv file. 
The cleaning taks are compile in a file dedicated, <a href="http://localhost:8888/notebooks/Documents/GitHub/DMML2019_Team_Tissot/code/DataCleaning.ipynb">Cleaning Notebook</a>.  
The pre-processin tasks are compiled in a notebook dedicated, the <a href="http://localhost:8888/notebooks/Documents/GitHub/DMML2019_Team_Tissot/code/Data%20Pre-processing.ipynb">Pre-processing Notebook</a>.   
  
Here is the link to a little <a href="https://www.youtube.com/watch?v=Ly7VEcCTke4&t=2s">video</a> compiling the notebook.




In [1]:
!pip install imblearn -q
!pip install keras -q
!pip install tensorflow -q
!pip install folium -q
!pip install seaborn -q

# Get the data
Same as in EDA notbook

In [2]:
import pandas as pd
import folium
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import re
import seaborn as sns
sns.set_style('white')
sns.set_color_codes("dark")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
df = pd.read_csv("../data/data_clean_new.csv", 
                          encoding='utf_8', 
                          dtype = 'unicode',
                          parse_dates = True,
                          infer_datetime_format = True,
                          low_memory=False)
df = df.drop("Unnamed: 0", axis = 1)

# Preprocess data

In [4]:
df.head(3)

,address,business_id,city,is_open,latitude,longitude,name,postal_code,review_count,stars,...,Sushi Bars,Tex-Mex,Thai,Vegan,Vegetarian,Vietnamese,Wine & Spirits,Wine Bars,Anymusic,name_length
0,30 Eglinton Avenue W,QXAEGFB4oINsVuTFxEYKFQ,Mississauga,1,43.6054989743,-79.652288909,Emerald Chinese Restaurant,L5R 3E7,0.014979029358897545,2.5,...,0,0,0,0,0,0,0,0,False,26
1,"10110 Johnston Rd, Ste 15",gnKjwL_1w79qoiV3IC_xQQ,Charlotte,1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,0.020011983223487118,4.0,...,1,0,0,0,0,0,0,0,False,27
2,2450 E Indian School Rd,1Dfx3zM-rW4n-31KeC8sJg,Phoenix,1,33.4951941,-112.0285876,Taco Bell,85016,0.0017974835230677054,3.0,...,0,1,0,0,0,0,0,0,False,9


In [5]:
df.stars.value_counts()

3.5    12833
4.0    12816
3.0     9421
4.5     5881
2.5     5080
2.0     2771
1.5      936
5.0      931
1.0      217
Name: stars, dtype: int64

In [6]:
#convert columns to the boolean datatype when we find the vlaue "True" or "False" in the column
for column in df.columns :
    if df[column][0] == 'True' or df[column][0] =='False':
        df[column] = df[column]=='True'
    #Often we find a True or False in the first line
    #This tests if the column is a boolean by using the first row for efficiency    
        
    #otherwise we test if we find a True or False value in the whole column
    elif "True" in df[column].values :
        df[column] = df[column]=='True'
    elif "False" in df[column].values :
        df[column] = df[column]=='True'

In [7]:
cuisine_type = ["American (New)","American (Traditional)","Arts & Entertainment","Asian Fusion","Bakeries","Barbeque","Bars",
"Beer","Breakfast & Brunch","Buffets","Burgers","Cafes","Canadian (New)","Caribbean","Caterers","Chicken Wings",
"Chinese","Cocktail Bars","Coffee & Tea","Comfort Food","Delis","Desserts","Diners","Ethnic Food",
"Event Planning & Services","Fast Food","Food","Food Delivery Services","French","Gastropubs","Gluten-Free",
"Greek","Grocery","Halal","Hot Dogs","Ice Cream & Frozen Yogurt","Indian","Italian","Japanese","Juice Bars & Smoothies",
"Korean","Latin American","Lounges","Mediterranean","Mexican","Middle Eastern","Nightlife","Pizza","Pubs",
"Salad","Sandwiches","Seafood","Soup","Specialty Food","Sports Bars","Steakhouses","Sushi Bars","Tex-Mex",
"Thai","Vegan","Vegetarian","Vietnamese","Wine & Spirits","Wine Bars"]

In [8]:
#The cuisine types have 1 or 0 instead of True/False
for column in df[cuisine_type] :
    df[column] = df[column]=="1"

In [9]:
df.columns

Index(['address', 'business_id', 'city', 'is_open', 'latitude', 'longitude',
       'name', 'postal_code', 'review_count', 'stars', 'state', 'Price',
       'American (New)', 'American (Traditional)', 'Arts & Entertainment',
       'Asian Fusion', 'Bakeries', 'Barbeque', 'Bars', 'Beer',
       'Breakfast & Brunch', 'Buffets', 'Burgers', 'Cafes', 'Canadian (New)',
       'Caribbean', 'Caterers', 'Chicken Wings', 'Chinese', 'Cocktail Bars',
       'Coffee & Tea', 'Comfort Food', 'Delis', 'Desserts', 'Diners',
       'Ethnic Food', 'Event Planning & Services', 'Fast Food', 'Food',
       'Food Delivery Services', 'French', 'Gastropubs', 'Gluten-Free',
       'Greek', 'Grocery', 'Halal', 'Hot Dogs', 'Ice Cream & Frozen Yogurt',
       'Indian', 'Italian', 'Japanese', 'Juice Bars & Smoothies', 'Korean',
       'Latin American', 'Lounges', 'Mediterranean', 'Mexican',
       'Middle Eastern', 'Nightlife', 'Pizza', 'Pubs', 'Salad', 'Sandwiches',
       'Seafood', 'Soup', 'Specialty Food', 'Spo

In [10]:
df['stars']=df['stars'].astype('float')
df.Price = pd.to_numeric(df.Price, errors='coerce')
df = df[np.isfinite(df['Price'])]
#df["Price"]= df["Price"].astype(int)

df["review_count"]= df["review_count"].astype(int)
df["name_length"]= df["name_length"].astype(int)
df["name"]= df["name"].astype(str)
df["address"]= df["address"].astype(str)

df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

ValueError: invalid literal for int() with base 10: '0.014979029358897545'

In [ ]:
df.dtypes

# EDA

### Heat map
Let's start by visualizing where the restaurants in our dataset are located

In [ ]:
from folium import plugins
from folium.plugins import HeatMap


# Make an empty map
m = folium.Map(location=[28,-90], tiles="OpenStreetMap", zoom_start=4)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = df[['latitude', 'longitude']]
heat_df = heat_df.dropna(axis=0, subset=['latitude','longitude'])

# List comprehension to make out list of lists
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(m)

# show the map
m

We can see that our data is only restaurants from North America.

### Stars on yelp

This will be the value that we want to predict, let's take a look.

In [ ]:
df.stars.describe()

In [ ]:
box_plot_data = df['stars'].astype("float")
plt.boxplot(box_plot_data)
plt.show()

We can see that we have an outlier, the 1 star ratings.

In [ ]:
r1 = df[["business_id","stars"]].groupby(["stars"]).count()
r1.plot.bar(x=None, 
            y=None,
            figsize=(15,5), 
            alpha = 0.8, # make the plot 20% transparent
            legend = None, 
           )

### Price
We think that this will be an important feature so let's get som insights

In [ ]:
ax = df.hist(column="stars", by='Price',bins=9, grid=False, figsize=(10,12), layout=(2,2), sharex=True, zorder=2, rwidth=0.9)

On this graph we ploted the restaurants price classes (from 1 to 4) by stars.
We can see the different distributions of ratings depending on the price category.

### Cuisine types

In [ ]:
r2 = df[cuisine_type].sum()
r2.plot.bar(x=None, y=None, figsize = (15, 5))

Since we have a lot of cuisine type we will analyze the top 20
We also remove the first 3 cuisine types (Food, Bars and Nightlife) since we don't

In [ ]:
top20_cuisines = list(df[cuisine_type].sum().sort_values(ascending=False).index[0:19])
top20_cuisines

In [ ]:
df2 = df.copy() #create new DataFrame
st = df2.loc[:, top20_cuisines].stack()

all_ids = pd.Series(st.index.get_level_values(1), 
                          st.index.get_level_values(0),
                          name='top cuisines')[st.values]

df2 = df2.join(all_ids, how='left').dropna()

In [ ]:
plt.figure(figsize = (10, 10))
sns.boxplot(data = df2[["top cuisines", "stars"]], x= "stars", y="top cuisines", color=matplotlib.colors.to_hex('#7479e8'))

We can see that Fast Food, Chicken Wings and Burgers have a lower star ratings median.
Cafes have have an above average median.

In [ ]:
df2.stars = df2.stars.astype(float)
avg_ratings_cuisine = pd.DataFrame(df2.groupby("top cuisines")["stars"].mean())
avg_ratings_cuisine

In [ ]:
sns.barplot(data= avg_ratings_cuisine, x = "stars", y= avg_ratings_cuisine.index )

### Base rate
The base rate is the size of the most common class divided by the size of the dataset.
Our accuracy should be better than the default rate

In [ ]:
df.stars.dtypes

In [ ]:
print("The most common class for the ratings is", df["stars"].mode()[0])

baseRate = df[df["stars"] == 3.5].count()["stars"] / df["stars"].count()
print("The baserate is :", baseRate)

# Preprocess data for models

The outlier we identified earlier, the 1 star rating, does not contain enough restaurants (only 216), thus we decided to drop it. We also drop string columns, which will not help in prediction (They are mainly related to geography).

In [ ]:
from sklearn import preprocessing
from sklearn import utils

In [ ]:
df.columns

In [ ]:
X = df[df['stars'] != 1.0]
X = X.drop(['stars', "name", "address", "business_id", "city", "state", 'postal_code', 'latitude', 'longitude'], axis = 1)
y = df[df["stars"] != 1.0]["stars"]

In [ ]:
y.value_counts()

In [ ]:
#SMOTE does not handle categorical data, we could also use SMOTE-NC


In [ ]:
X.Anymusic = X.Anymusic.astype(bool)
X.is_open = X.is_open.astype(bool)

In [ ]:
lab_enc = preprocessing.LabelEncoder()
encoded_y = lab_enc.fit_transform(y) #we label encode the star ratings
#X.Price = lab_enc.fit_transform(X.Price)

We divide our data into train and test.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from pprint import pprint
from time import time

In [ ]:
# split train/test
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.2, random_state=72)

Now, we decided to try two methods to fight severe class imbalance. Just downsampling is not an option (then each class will have 900 observations), thus, first we tried upsampling our train data and the we wanted to try to combine the two methods by upsampling all the classes below the mean and downsampling the classes above it. 

For upsampling we used a technique called SMOTE (**S**ynthetic **M**inority **O**ver-sampling **TE**chnique) that will synthesize new minority instances. So in our case we will basically generate "fake" restaurants based on other on the data set we have, for more details check out : http://rikunert.com/SMOTE_explained .

For downsampling we used the NearMiss methode

"first, the method calculates the distances between all instances of the majority class and the instances of the minority class. Then k instances of the majority class that have the smallest distances to those in the minority class are selected. If there are n instances in the minority class, the “nearest” method will result in k*n instances of the majority class."

source : https://towardsdatascience.com/sampling-techniques-for-extremely-imbalanced-data-part-i-under-sampling-a8dbc3d8d6d8

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [ ]:
unique, counts = np.unique(encoded_y, return_counts=True)
dict(zip(unique, counts))

In [ ]:
#upsamples all the train data
X_resampled, y_resampled = SMOTE(random_state = 72).fit_resample(X_train, y_train)

In [ ]:
med_cl_cnt= int(pd.Series(y_train).value_counts().median()) # median of obs over classes
med_cl_cnt

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

In [ ]:
#downsamples and upsamples the train data (based on median)
X_resampled2, y_resampled2 = NearMiss(sampling_strategy = {3:med_cl_cnt, 4:med_cl_cnt, 5:med_cl_cnt, 6:med_cl_cnt}).fit_resample(X_train, y_train) #downsampling
X_resampled2, y_resampled2 = SMOTE(random_state = 72, sampling_strategy = {7:med_cl_cnt, 0:med_cl_cnt, 1:med_cl_cnt, 2:med_cl_cnt}).fit_resample(X_resampled2, y_resampled2) # upsampling

In [ ]:
unique, counts = np.unique(y_resampled2, return_counts=True)
dict(zip(unique, counts))

Specify parameters values for grid search: 

In [ ]:
parametersRF = {
    'n_estimators': (100,200,300),
    'max_depth': (10,20,30)
}
parametersLR = {
    'C': (0.1, 1,100),
    'solver': (['saga','lbfgs'])
}
parametersNN = {
    'epochs': ([10, 100]),
    'batch_size': ([20,30])
}

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

We tried three different samples of data: normal, upsampled, upsampled+downsampled 

We found that for logistic regression the best result is produced with the normal data

In [ ]:
# decomment, if need to run best model with other params 
LR = LogisticRegression(solver='lbfgs',C=0.1, max_iter=2000, multi_class = "auto")
LR.fit(X_train, y_train)

In [ ]:
if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(LR, parametersLR, cv=2,
                               n_jobs=-1, verbose=1,scoring='accuracy')

    print("Performing grid search...")
    #print("pipeline:", [name for name, _ in pipeline2.steps])
    print("parameters:")
    pprint(parametersLR)
    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score for Logistic Regression: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parametersLR.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

The best model according to gridsearch is the one we tested at the beginning with lbfgs solver and C equal to 0.1

In [ ]:
# train accuracy
LR.score(X_train, y_train)

In [ ]:
# test accuracy
LR.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report
target_names = ["1.5","2","2.5","3","3.5","4","4.5","5"]

print(classification_report(y_test, LR.predict(X_test), target_names= target_names))

Our test accuracy is above the baserate but it isn't really a good result since we cannot predict 1.5 and 5 stars.

# Random Forest Classifier

Here we will try both data samples, but still we believe that sticking to one method of upsampling is better (at least more common).

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
MAE = make_scorer(mean_absolute_error)
folds = 3

In [ ]:
clf = RandomForestClassifier(n_estimators = 200,max_depth = 30)
clf.fit(X_resampled, y_resampled)

In [ ]:
if __name__ == "__main__": # just for multiprocessing purposes
    grid_search = GridSearchCV(clf, parametersRF, cv=3,
                               n_jobs=-1, verbose=1,scoring='accuracy')

    print("Performing grid search...")
    #print("Random Forest:", [name for name, _ in clf.steps])
    print("parameters:")
    pprint(parametersRF)
    t0 = time()
    grid_search.fit(X_resampled2, y_resampled2)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score for Random Forest: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parametersRF.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
clf = RandomForestClassifier(n_estimators = 300,max_depth = 30)
clf.fit(X_resampled, y_resampled)

In [ ]:
clf2 = RandomForestClassifier(n_estimators = 300,max_depth = 30)
clf2.fit(X_resampled2, y_resampled2)

In [ ]:
print("Train accuracy :", clf.score(X_resampled, y_resampled))
print("Test accuracy :", clf.score(X_test, y_test))

In [ ]:
print("Train acc upsample + downsample :", clf2.score(X_resampled2, y_resampled2))
print("Test acc upsample + downsample :", clf2.score(X_test, y_test))

Report for upsampling:

In [ ]:
print(classification_report(y_test, clf.predict(X_test), target_names= target_names))

Report for upsampling + downsampling:

In [ ]:
print(classification_report(y_test, clf2.predict(X_test), target_names= target_names)) 

For random forest upsampling data produces the best result

At the end, we would like to use mean absolute error as a performance metric for our models, because predicting rating of the restaurant is a classification with ordinal variable. Thus, misclassification of 0.5 star is better than 1.5 stars.

In [ ]:
MAE_RF = cross_val_score(clf,
    X_resampled,
    y_resampled,
    cv=folds,
    scoring=MAE)

In [ ]:
print("Random forest MAE :", np.mean(MAE_RF))

In [ ]:
model = ExtraTreesClassifier(n_estimators=100)
model.fit(X_resampled, y_resampled)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

# Neural network

Here we are implementing neural network with basic architecture on not sampled data. 

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
np.random.seed(1143)

In [ ]:
def model_NN():
    model = Sequential()
    model.add(Dense(512, input_shape=(69,)))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit"
                              # we clamp all values below 0 to 0. 
            
    model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(8))
    model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.
    #optimizer = optimizers.Adam(lr=0.01, decay=1e-6)
    optimizer = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn= model_NN, epochs=100, batch_size=10, verbose=0)

In [ ]:
X = df[df["stars"] != 1]
X = X.drop(['stars', "name", "address", "business_id", "city", "state", 'postal_code', "longitude", "latitude"], axis = 1)
y = df[df["stars"] !=1]["stars"]

In [ ]:
#nb_classes = len(y.value_counts())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)


In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)

In [ ]:
model_hist = model.fit(X_train, y_train,
                       batch_size=64, epochs=10,
                       verbose=1, validation_split=0.2)

In [ ]:
score = model.score(X_test, y_test, verbose=0)
score

## Conclusion

We conducted good EDA for YELP restaurant data and tested three different models to predict the star rating.

The best model in terms of stars prediction turned out to be neural network with test accuracy equal to 0.43.

Thus, we suggest our friends use this model to understand, how their current restaurant performance will behave in America and estimate their rating among clients.